# Run Plain Simulation
* Constant frequency $f$, Packing $\Phi$, and height $h$. 
* Varying tilt angle $\delta$. 

Saved in 

    ../Data/FirstRun

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st

from tqdm.auto import tqdm

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "../Data/Run1-PlainRun"

os.system("rm -r %s"%directory)

0

In [3]:
h = 3.9*ureg.um
rho = 1.4 * ureg.um

region, positions = mgc.ordered_dimers(
    n_of_particles = 250,
    packing=0.35, height = h.magnitude,
    r = rho.magnitude, theta = 0)

particles = mgc.particles(
    positions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [4]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [5]:
world = mgc.world(particles, temperature = 300*ureg.K,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                  region=region*ureg.um, boundaries = ['p','p','f'], 
                  walls = [False,False,True],
                  dipole_cutoff = 30*ureg.um)

total_time = 60*ureg.s
tilt_angle = np.arange(0,30,1)

In [6]:
index_name = "index.dat"

import csv

def prepare_simulation(delta):
    
    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 3*ureg.Hz,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,delta*ureg.deg)

    sim = mgc.sim(dir_name = directory,
                  file_name="Tilt_%s_deg_h%u"%(delta,h.to(ureg.nm).magnitude),stamp_time=True,
                  timestep = 1e-4*ureg.s, framerate = 60*ureg.Hz, total_time = total_time,
                  particles = particles, world = world, 
                  field = field, output=["x","y","z","mux","muy","muz","fx","fy"])
    
    sim.tilt = delta
    sim.height = h
    
    return sim

In [7]:
simulations = [prepare_simulation(delta) for delta in tilt_angle[:]]

In [8]:
def run_simulation(sim):
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
    
    sim.generate_scripts()
    sim.run()
    
    sim.load()
    max_frame = max(sim.lazy_read.T.keys())
    
    h = sim.height
    delta = sim.tilt
    
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1], h.magnitude, delta, max_frame])

## Run Asynchronously

In [9]:
from multiprocessing import Pool

In [10]:
%%time
if __name__ ==  '__main__': 
    num_processors = 15
    p=Pool(processes = num_processors)
    p.map(run_simulation,simulations)

CPU times: user 36.3 ms, sys: 67.1 ms, total: 103 ms
Wall time: 47min 33s


# Show some results

In [11]:
index = pd.read_csv(os.path.join(directory,"index.dat"),
                    names = ["file", "height", "tilt", "max_frame"],
                    sep=" ")

In [12]:
index.head()

,file,height,tilt,max_frame
0,Tilt_9_deg_h3899_2023_07_23_18_07_28,3.9,9,599864
1,Tilt_3_deg_h3899_2023_07_23_18_07_28,3.9,3,599864
2,Tilt_10_deg_h3899_2023_07_23_18_07_28,3.9,10,599864
3,Tilt_7_deg_h3899_2023_07_23_18_07_28,3.9,7,599864
4,Tilt_2_deg_h3899_2023_07_23_18_07_28,3.9,2,599864


In [13]:
name = index.query("tilt == 0").iloc[0].file

In [14]:
trj_read = mgc.trj_lazyread(os.path.join(directory,name+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])
trj = trj_read[:]
bounds = trj_read.get_bounds()

In [15]:
trj.head()

type          x          y         z           mux      muy  \
frame id                                                                
0     1      1 -28.963200 -30.360701  0.561224  8.895310e-07  0.00004   
      2      1 -31.590700 -30.360701 -0.561224  8.480380e-07  0.00004   
      3      1 -22.589100 -30.360701  0.561224  9.409800e-07  0.00004   
      4      1 -25.216600 -30.360701 -0.561224  9.336550e-07  0.00004   
      5      1 -16.215099 -30.360701  0.561224  9.496160e-07  0.00004   

               muz   fx   fy  
frame id                      
0     1   0.000077  0.0  0.0  
      2   0.000077  0.0  0.0  
      3   0.000077  0.0  0.0  
      4   0.000077  0.0  0.0  
      5   0.000077  0.0  0.0

In [16]:
frames = trj.index.get_level_values("frame").unique()
anim = mgc.animate_trj(trj.loc[frames[:100]],
                       region = bounds.iloc[0], radius = 1.4, framerate = 20, verb = False)

In [17]:
vid = anim.to_html5_video()

In [18]:
HTML(vid)

### tilted result

In [19]:
index = pd.read_csv(os.path.join(directory,"index.dat"),
                    names = ["file", "height", "tilt", "max_frame"],
                    sep=" ")

In [20]:
index.head()

,file,height,tilt,max_frame
0,Tilt_9_deg_h3899_2023_07_23_18_07_28,3.9,9,599864
1,Tilt_3_deg_h3899_2023_07_23_18_07_28,3.9,3,599864
2,Tilt_10_deg_h3899_2023_07_23_18_07_28,3.9,10,599864
3,Tilt_7_deg_h3899_2023_07_23_18_07_28,3.9,7,599864
4,Tilt_2_deg_h3899_2023_07_23_18_07_28,3.9,2,599864


In [21]:
name = index.query("tilt == 10").iloc[0].file

In [22]:
name

'Tilt_10_deg_h3899_2023_07_23_18_07_28'

In [23]:
trj_read = mgc.trj_lazyread(os.path.join(directory,name+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])
trj = trj_read[:]
bounds = trj_read.get_bounds()

In [24]:
trj.head()

type          x          y         z       mux      muy       muz  \
frame id                                                                      
0     1      1 -28.963200 -30.360701  0.561224  0.000015  0.00004  0.000076   
      2      1 -31.590700 -30.360701 -0.561224  0.000015  0.00004  0.000076   
      3      1 -22.589100 -30.360701  0.561224  0.000015  0.00004  0.000076   
      4      1 -25.216600 -30.360701 -0.561224  0.000015  0.00004  0.000076   
      5      1 -16.215099 -30.360701  0.561224  0.000015  0.00004  0.000076   

           fx   fy  
frame id            
0     1   0.0  0.0  
      2   0.0  0.0  
      3   0.0  0.0  
      4   0.0  0.0  
      5   0.0  0.0

In [25]:
frames = trj.index.get_level_values("frame").unique()
anim = mgc.animate_trj(trj.loc[frames[:100]], region = bounds.iloc[0], radius = 1.4, framerate = 20, verb = False)

In [26]:
vid = anim.to_html5_video()

In [27]:
HTML(vid)

In [28]:
index

,file,height,tilt,max_frame
0,Tilt_9_deg_h3899_2023_07_23_18_07_28,3.9,9,599864
1,Tilt_3_deg_h3899_2023_07_23_18_07_28,3.9,3,599864
2,Tilt_10_deg_h3899_2023_07_23_18_07_28,3.9,10,599864
3,Tilt_7_deg_h3899_2023_07_23_18_07_28,3.9,7,599864
4,Tilt_2_deg_h3899_2023_07_23_18_07_28,3.9,2,599864
5,Tilt_11_deg_h3899_2023_07_23_18_07_28,3.9,11,599864
6,Tilt_0_deg_h3899_2023_07_23_18_07_28,3.9,0,599864
7,Tilt_1_deg_h3899_2023_07_23_18_07_28,3.9,1,599864
8,Tilt_12_deg_h3899_2023_07_23_18_07_28,3.9,12,599864
9,Tilt_8_deg_h3899_2023_07_23_18_07_28,3.9,8,599864
